In [ ]:
!pip -q install mlxtend python-slugify tqdm


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving consumption_user.csv to consumption_user.csv


In [ ]:
import pandas as pd
import zipfile
from pathlib import Path
import os

def read_first_csv_from_zip(zip_path: str) -> pd.DataFrame:
    with zipfile.ZipFile(zip_path, "r") as z:
        csv_names = [n for n in z.namelist() if n.lower().endswith(".csv")]
        if not csv_names:
            raise FileNotFoundError("Nu exista CSV în ZIP.")


        def score(name: str):
            lower = name.lower()
            s = 0
            if "consumption" in lower: s += 3
            if "food" in lower: s += 2
            if "intake" in lower: s += 2
            info = z.getinfo(name)
            return (s, info.file_size)

        best = sorted(csv_names, key=score, reverse=True)[0]
        print("Am ales din ZIP:", best)
        with z.open(best) as f:
            return pd.read_csv(f, low_memory=False)

def load_any(input_path: str) -> pd.DataFrame:
    p = Path(input_path)
    if not p.exists():
        raise FileNotFoundError(f"Nu gasesc: {p}")

    if p.suffix.lower() == ".zip":
        return read_first_csv_from_zip(str(p))
    elif p.suffix.lower() in [".csv", ".tsv"]:
        sep = "\t" if p.suffix.lower() == ".tsv" else ","
        return pd.read_csv(p, sep=sep, low_memory=False)
    else:
        raise ValueError("Accept doar .zip/.csv/.tsv")


if 'uploaded' in globals() and len(uploaded) > 0:
    INPUT_PATH = list(uploaded.keys())[0]

df_raw = load_any(INPUT_PATH)
print("Shape:", df_raw.shape)
df_raw.head()


Shape: (257952, 80)


,SUBJECT,ROUND,SURVEY_DAY,RESPONDER,SEASON,CONSUMPTION_DAY,CONSUMPTION_MONTH,CONSUMPTION_YEAR,WEEK_DAY,EXCEPTION_DAY,...,RETOL_mcg,VITD_mcg,VITE_mg,VITK_mcg,PHY_mg,N6_g,EPA_DHA_g,ALA_g,EAT_SEQ,VERSION
0,1,1,1,1,NaN,NaN,NaN,2012,2,1,...,NaN,1.6,0.35,5.12,NaN,3.50,0.0,0.20,1,16-Jun-21
1,1,1,1,1,NaN,NaN,NaN,2012,2,1,...,NaN,0.0,0.57,11.55,NaN,1.32,0.0,0.12,1,NaN
2,1,1,1,1,NaN,NaN,NaN,2012,2,1,...,NaN,0.0,0.93,2.80,NaN,0.87,0.0,0.13,1,NaN
3,1,1,1,1,NaN,NaN,NaN,2012,2,1,...,NaN,0.0,0.00,0.00,NaN,NaN,0.0,NaN,2,NaN
4,1,1,1,1,NaN,NaN,NaN,2012,2,1,...,NaN,0.0,0.03,60.68,NaN,NaN,0.0,NaN,1,NaN


In [ ]:
import re

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def detect_columns(df: pd.DataFrame):
    cols = list(df.columns)
    ncols = [_norm(c) for c in cols]

    def find_any(patterns):
        for pat in patterns:
            for c, nc in zip(cols, ncols):
                if re.search(pat, nc):
                    return c
        return None

    person = find_any([r"personid", r"individualid", r"subjectid", r"respondentid", r"pid", r"indid"])
    day    = find_any([r"dayid", r"recallid", r"recordid", r"surveyday", r"visit", r"day"])
    food   = find_any([r"fooditem", r"foodname", r"food", r"subgroup", r"foodgroup", r"item"])
    amount = find_any([r"amount", r"gram", r"edible", r"consumed", r"intake", r"g$"])

    return {"person": person, "day": day, "food": food, "amount": amount}

cols = detect_columns(df_raw)
cols
PERSON = "SUBJECT"
DAY    = "SURVEY_DAY"
FOOD   = "FOOD_TYPE"
AMOUNT = "AMOUNT_RECIPE"

PERSON, DAY, FOOD, AMOUNT



('SUBJECT', 'SURVEY_DAY', 'FOOD_TYPE', 'AMOUNT_RECIPE')

In [ ]:
import numpy as np


PERSON = "SUBJECT"
DAY    = "SURVEY_DAY"
FOOD = "FOODEX2_RECIPE_DESCR"
AMOUNT = "AMOUNT_RECIPE"


for c in [PERSON, DAY, FOOD, AMOUNT]:
    assert c in df_raw.columns, f"Lipseste coloana: {c}"
print("OK coloane:", PERSON, DAY, FOOD, AMOUNT)

df = df_raw.copy()
df = df.dropna(subset=[PERSON, DAY, FOOD])

# pastram doar consum > 0
df[AMOUNT] = pd.to_numeric(df[AMOUNT], errors="coerce").fillna(0.0)
df = df[df[AMOUNT] > 0]

df[FOOD] = df[FOOD].astype(str).str.strip()

# tranzactie = persoana-zi
df["txn_id"] = df[PERSON].astype(str) + "_" + df[DAY].astype(str)

transactions = (
    df.groupby("txn_id")[FOOD]
      .apply(lambda s: sorted(set(s)))
      .tolist()
)

print("Exemplu tranzactie:", transactions[0][:20] if transactions else "EMPTY")
print("Nr tranzactii:", len(transactions))



OK coloane: SUBJECT SURVEY_DAY FOODEX2_RECIPE_DESCR AMOUNT_RECIPE
Exemplu tranzactie: ['Mixed vegetables soup', 'Potato based dishes', 'Rice, meat, and vegetables meal']
Nr tranzactii: 9143


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
import time

# One-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
X = pd.DataFrame(te_ary, columns=te.columns_)

print("Nr itemi (alimente distincte):", X.shape[1])

# Parametri
min_support = 0.015
min_conf    = 0.6
min_lift    = 1.2

# FP-Growth
t0 = time.perf_counter()
fi_fp = fpgrowth(X, min_support=min_support, use_colnames=True)
t_fp = time.perf_counter() - t0

# Apriori
t0 = time.perf_counter()
fi_ap = apriori(X, min_support=min_support, use_colnames=True)
t_ap = time.perf_counter() - t0

print(f"FP-Growth: itemsets={len(fi_fp)} | time={t_fp:.3f}s")
print(f"Apriori:   itemsets={len(fi_ap)} | time={t_ap:.3f}s")

# Reguli (din FP-Growth)
rules = association_rules(fi_fp, metric="confidence", min_threshold=min_conf)
rules = rules[rules["lift"] >= min_lift].copy()


rules["antecedents"] = rules["antecedents"].apply(lambda x: ", ".join(sorted(list(x))))
rules["consequents"] = rules["consequents"].apply(lambda x: ", ".join(sorted(list(x))))

rules = rules.sort_values(["lift", "confidence", "support"], ascending=False)

rules[["antecedents", "consequents", "support", "confidence", "lift"]].head(15)


Nr itemi (alimente distincte): 80
FP-Growth: itemsets=48 | time=0.644s
Apriori:   itemsets=48 | time=0.018s


,antecedents,consequents,support,confidence,lift


In [ ]:
from mlxtend.frequent_patterns import association_rules


rules_all = association_rules(fi_fp, metric="confidence", min_threshold=0.01)

print("Reguli totale generate:", len(rules_all))
if len(rules_all) > 0:
    display(rules_all[["antecedents","consequents","support","confidence","lift"]].head(10))


min_conf = 0.3
min_lift = 1.0

rules_fp = rules_all[(rules_all["confidence"] >= min_conf) & (rules_all["lift"] >= min_lift)].copy()
rules_fp = rules_fp.sort_values(["lift","confidence","support"], ascending=False)


rules_fp["antecedents"] = rules_fp["antecedents"].apply(lambda x: ", ".join(sorted(list(x))))
rules_fp["consequents"] = rules_fp["consequents"].apply(lambda x: ", ".join(sorted(list(x))))

print("Reguli dupa filtrare:", len(rules_fp))
rules_fp[["antecedents","consequents","support","confidence","lift"]].head(20)




Reguli totale generate: 44


,antecedents,consequents,support,confidence,lift
0,(Potato based dishes),(Mixed vegetables soup),0.029312,0.298774,1.076315
1,(Mixed vegetables soup),(Potato based dishes),0.029312,0.105595,1.076315
2,(Potato based dishes),(Mixed vegetable salad),0.030734,0.313266,1.173850
3,(Mixed vegetable salad),(Potato based dishes),0.030734,0.115164,1.173850
4,(Meat and vegetable soup),(Potato based dishes),0.023406,0.103382,1.053755
5,(Potato based dishes),(Meat and vegetable soup),0.023406,0.238573,1.053755
6,(Meat based dishes),(Potato based dishes),0.018047,0.136026,1.386499
7,(Potato based dishes),(Meat based dishes),0.018047,0.183946,1.386499
8,(Mixed vegetables soup),"(Rice, meat, and vegetables meal)",0.018156,0.065406,0.995017
9,"(Rice, meat, and vegetables meal)",(Mixed vegetables soup),0.018156,0.276206,0.995017


Reguli dupa filtrare: 7


,antecedents,consequents,support,confidence,lift
37,Vegetable based dishes,Mixed vegetables soup,0.016406,0.370370,1.334238
33,Potatoes and vegetables meal,Mixed vegetable salad,0.019359,0.313830,1.175961
2,Potato based dishes,Mixed vegetable salad,0.030734,0.313266,1.173850
42,"Omelette, plain",Mixed vegetable salad,0.015312,0.313199,1.173598
38,"Mixed vegetables, fried",Mixed vegetables soup,0.016515,0.318565,1.147614
40,"Omelette, plain",Mixed vegetables soup,0.015422,0.315436,1.136341
11,"Rice, meat, and vegetables meal",Mixed vegetable salad,0.019906,0.302829,1.134739


In [ ]:
import os
from pathlib import Path

out_dir = Path("/content/data/output")
out_dir.mkdir(parents=True, exist_ok=True)

fi_fp.to_csv(out_dir/"frequent_itemsets_fpgrowth.csv", index=False)
fi_ap.to_csv(out_dir/"frequent_itemsets_apriori.csv", index=False)

rules_fp.to_csv(out_dir/"rules_fpgrowth.csv", index=False)

benchmark = pd.DataFrame([{
    "min_support": min_support,
    "fp_itemsets": len(fi_fp),
    "fp_time_sec": t_fp,
    "ap_itemsets": len(fi_ap),
    "ap_time_sec": t_ap
}])
benchmark.to_csv(out_dir/"benchmark.csv", index=False)

print("Salvat în:", out_dir)
os.listdir(out_dir)


Salvat în: /content/data/output


['frequent_itemsets_apriori.csv',
 'frequent_itemsets_fpgrowth.csv',
 'rules_fpgrowth.csv',
 'benchmark.csv']

In [ ]:
# Top itemi (alimente/grupe) dupa frecventa
item_support = X.mean().sort_values(ascending=False)
item_support.head(20).to_frame("support")


,support
Mixed vegetables soup,0.277589
Mixed vegetable salad,0.266871
Meat and vegetable soup,0.226403
Meat based dishes,0.132670
Potato based dishes,0.098108
Meat soup,0.093952
"Pasta, cooked, with cheese/cream",0.071311
Beans and vegetables meal,0.066171
"Rice, meat, and vegetables meal",0.065733
Potatoes and vegetables meal,0.061687
